<!--NOTEBOOK_HEADER-->
*This notebook contains material from [Sparsity in optimal randomized classification trees](https://idus.us.es/handle/11441/107840).

## Imports

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import scipy.stats as stats
import scipy.optimize as optimize

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt 
        except:
            pass

assert(shutil.which("ipopt") or os.path.isfile("ipopt"))

from pyomo.environ import *
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

     |████████████████████████████████| 9.7 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 2.0 MB/s 


## Description

This notebook uses [Pyomo](http://www.pyomo.org/) to implement the sparsity in optimal randomized classification trees model (SORCT) and tests it using [The Iris Dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) according to the the paper [Sparsity in optimal randomized classification trees](https://idus.us.es/handle/11441/107840). The SORCT based on oblique cuts proposes a continuous optimization approach for both local and global sparsity, where all decision branches are optimised simultaneously, and has demonstrated a more thorough performance when compared to traditional decision trees like CARTs using the greedy algorithm, which focuses on local sparsity.

## Decision Tree

From 2.2.1 of the [paper](https://idus.us.es/bitstream/handle/11441/107840/Sparsity%20in%20optimal%20randomized%20classification%20trees.pdf?sequence=1&isAllowed=y):

| *Parameters* <img width=50/> | <img width=250/> |
| :--------------------------- | :- |
| $D$ | depth of the binary tree |
| $N$ | number of individuals in the training sample |
| $p$ | number of predictor variables |
| $K$ | number of classes |
| $\{(xi, yi)\}_{1\leq i\leq N}$ | training sample, where $x_i ∈ [0, 1]^p$ and <br> $y_i \in \{1, . . ., K\}$ |
| $I_k$ | set of individuals in the training sample <br> belonging to class $k, k = 1, . . ., K$ |
| $W_{y_{i^k}}$ | misclassification cost incurred when <br> classifying an individual $i$, whose class is $y_i$ , <br> in class $k, yi, i = 1, . . ., N, k = 1, . . ., K$ |
| $F (·)$ | univariate continuous CDF centered at 0, used <br> to define the probabilities for an individual to <br> go to the left or the right child node in the <br> tree. We will assume that $F$ is the CDF of a <br> continuous random variable with density $f$ |
| $\lambda ^L \ge 0$ | local sparsity regularization parameter |
| $\lambda ^G \ge 0$ | global sparsity regularization parameter |

<br>

| *Nodes* <img width=77/> | <img width=250/> |
| :---------------------- | :- |
| $\tau _ B$ | set of branch nodes |
| $\tau _ L$ | set of leaf nodes |
| $N_L(t)$ | set of ancestor nodes of leaf node $t$ whose <br> left branch takes part in the path from the <br> root node to leaf node $t$, $t \in \tau$ |
| $N_L(t)$ | set of ancestor nodes of leaf node $t$ whose <br> right branch takes part in the path from the <br> root node to leaf node $t$, $t \in \tau _L$ |

<br>

| *Decision variables* <img width=15/> | <img width=250/> |
| :----------------------------------- | :- |
| $a_{jt} \in [−1, 1]$ | coefficient of predictor variable $j$ in the <br> oblique cut at branch node $t \in \tau _B$, with **$a$** <br> being the $p × \mid \tau _B \mid$ matrix of these coefficients <br> , $a = (a_{jt}) _{j=1,...,p, t∈τB}$. The expressions $a_j·$ <br> and **$a_t$** will denote the $j$-th row and the $t$-th <br> column of **$a$**, respectively |
| $\mu_t \in [−1, 1]$ | location parameter at branch node $t \in \tau_B$, $\mu$ <br> being the vector that comprises every $\mu_t$, i.e., <br> $\mu = (\mu_t)_{t\in \tau_B}$ |
| $C_{kt}$ | probability of being assigned to class label <br> $k \in {1, . . ., K}$ for an individual at leaf node <br> $t$, $t \in \tau_L$, being the $K × \mid \tau_L\mid$ matrix such that <br> $C = (C_{kt})_{k=1,...,K, t\in \tau_L}$ |

<br>

| *Probabilities* <img width=45/> | <img width=250/> |
| :------------------------------ | :- |
| $p_{it}($**$a$**$_{·t},\mu_t)$ | probability of individual $i$ going down the left <br> branch at branch node $t$. Its expression is <br> $p_{it}($**$a$**$_{·t},\mu_t) = F(\frac{1}{p}$**$a$**$^T_{·t}x_i-$**$\mu$**$_t)$, <br> $i = 1,...,N, t \in \tau_B$ |
| $P_{it}($**$a$**$,\bf{\mu})$ | probability of individual $i$ failing into leaf <br> node $t$. Its expression is <br> $P_{it}($**$a$**$,\mu) = \prod\limits_{t_l \in N_L(t)} p_{it_l}($**$a$**$_{·t_l},\mu_{t_l}) $<br>$\prod\limits_{t_l \in N_R(t)} (1 - p_{it_r}($**$a$**$_{·t_r},\mu_{t_r})) $, <br> $i = 1,...,N, t \in \tau_L$ |
| $g(a,\mu,C)$ | expected misclassification cost over the <br> training sample. Its expression is <br> $g(a,\mu,C) = \frac{1}{N}\sum\limits^N_{i=1}\sum\limits_{t \in \tau_L} P_{it}(a, \mu) \sum\limits^K_{k=1} W_{y_{i^k}}C_{kt}$ |

<br>

| *Objective* <img width=262/> | <img width=50/> |
| :--------------------------- | :- |
| min $g(a,\mu,C) + \lambda^L\sum\limits^p_{j=1}\|a_{j·}\|_1 + \lambda^G\sum\limits^p_{j=1}\|a_{j·}\|_\infty $ | |

<br>

| *Constraint* <img width=257/> | <img width=50/> |
| :---------------------------- | :- |
| $\sum\limits^K_{k=1} C_{kt} = 1, t \in \tau_L$ | |
| $\sum\limits_{t \in \tau_L} C_{kt} \ge 1, k=1,...,K$ | |
| $a_{jt} \in [-1,1], j=1,...,p, t\in \tau_B$ | |
| $\mu_t \in [-1,1], t\in \tau_B$ | |
| $C_{kt} \in [0,1], k=1,...,K, t\in \tau_L$ | |

<br>

Achieved using Pyomo below:


In [2]:
# optimal randomized classification trees model

def orct(X, y, depth, F_turn = 128, l_reg = 0.1, g_reg = 1):
    n_instances, n_features = X.shape
    
    m = ConcreteModel()

    # D depth
    m.D = Set(initialize=list(range(depth)))
    # p predictor variables #s
    m.p = Set(initialize=list(range(n_features)))
    # N training sample #s
    m.N = Set(initialize=list(range(n_instances)))
    # k categories
    m.k = Set(initialize=list(range(len(np.unique(y)))))
    # training samples
    train_set = {(n, p): X[n,p] for n in m.N for p in m.p}
    m.X = Param(m.N, m.p, within=Reals, initialize=train_set)
    m.y = Param(m.N, within=Reals, initialize={n: y[n] for n in m.N}) # category k 
    m.training_samples = Param(m.N, initialize={n: (X[n], y[n]) for n in m.N})
    # I_k
    idic = {}
    for j in m.training_samples:
        if m.training_samples[j][1] not in idic:
            idic[m.training_samples[j][1]] = [tuple(m.training_samples[j][0])]
        else:
            idic[m.training_samples[j][1]].append(tuple(m.training_samples[j][0]))
    m.Ik = Set(m.k,initialize=idic)
    # W_yik
    m.W_yik = Param(m.k, m.N, within=Binary, initialize={(k, n): int(m.y[n]!=k) for k in m.k for n in m.N})


    # F
    def F(x):
        return 1./(1. + exp(-x*F_turn))


    # Nodes
    m.tb = Set(initialize=list(range(1,2**depth)))
    m.tl = Set(initialize=list(range(2**depth,2*2**depth)))
    def Nl_Nr(node):
        Nl_list = []
        Nr_list = []
        while node>1:
            if node%2 == 0:
                node = node//2
                Nl_list.append(node)
            else:
                node = (node-1)//2
                Nr_list.append(node)
        return (Nl_list,Nr_list)
    m.Nl = Set(m.tl,initialize={n: Nl_Nr(n)[0] for n in m.tl})
    m.Nr = Set(m.tl,initialize={n: Nl_Nr(n)[1] for n in m.tl})


    # VARIABLES
    m.a_jt = Var(m.p, m.tb, bounds=[-1,1], within=Reals)
    m.a_jt_Max = Var(m.p, within=Reals) # max() for each vactor a_j.
    m.mu_t = Var(m.tb, bounds=[-1,1], within=Reals)
    m.C_kt = Var(m.k, m.tl, bounds=[0,1], within=Reals)


    # Probabilities
    def p_it(model, tb, i):
        return F((1/n_features) * (sum(model.a_jt[p,tb]*model.X[i, p] for p in model.p)) - model.mu_t[tb])
    m.p_it = Expression(m.tb, m.N, rule=p_it)

    def P_it(model, tl, i):
        return prod(p_it(model, nl, i) for nl in model.Nl[tl])*prod(1-p_it(model, nr, i) for nr in model.Nr[tl])
    m.P_it = Expression(m.tl, m.N, rule=P_it)

    def g(model, i, tl):
        return model.P_it[tl,i] * sum(model.W_yik[k,i]*model.C_kt[k,tl] for k in model.k)
    m.g = Expression(m.N, m.tl, rule=g)


    # Objective
    def cost_rule(model):
        cost = 0.0
        for i in model.N:
          for tl in model.tl:
            cost += model.g[i,tl]
        # return cost
        lreg = 0.0
        greg = 0.0
        for p in model.p:
          greg += model.a_jt_Max[p]
          for tb in model.tb:
            lreg += model.a_jt[p,tb]
        return cost+l_reg*lreg+g_reg*greg
    m.cost = Objective(rule=cost_rule, sense=minimize)


    # Constraint
    def aRule(model,tl):
        return sum(model.C_kt[k,tl] for k in model.k) == 1
    m.Boundk_C_kt = Constraint(m.tl, rule=aRule)

    def bRule(model,k):
        return sum(model.C_kt[k,tl] for tl in model.tl) >= 1
    m.Boundt_C_kt = Constraint(m.k, rule=bRule)

    def a_jt_Max_Rule(model, p, tb): # for global sparsity regularization parameter
        return model.a_jt_Max[p] >= model.a_jt[p, tb]
    m.Bound_a_jt_Max = Constraint(m.p, m.tb,rule=a_jt_Max_Rule)


    return m


# optimal randomized classification trees prediction function

def orct_predict(model, X, F_turn=128):
  n_instances, n_features = X.shape
  def F(x):
        return 1./(1. + exp(-x*F_turn))

  P_it = np.zeros((len(model.tb)+len(model.tl), n_instances))
  for tb in model.tb:
    for i in range(n_instances):
      P_it[tb-1, i] = F((1/n_features) * (sum(model.a_jt[p,tb]()*X[i, p] for p in model.p)) - model.mu_t[tb]())
  for tl in model.tl:
    for i in range(n_instances):
      P_it[tl-1, i] = prod(P_it[nl-1, i] for nl in model.Nl[tl])*prod(1-P_it[nr-1, i] for nr in model.Nr[tl])
  return [np.argmax([sum(P_it[tl-1, n]*model.C_kt[cl, tl]() for tl in model.tl) for cl in model.k]) for n in range(n_instances)]

Load the Iris dataset.

In [3]:
# Iris dataset
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

Solving by ipopt.

In [4]:
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

orct_model = orct(X_train, y_train, depth=3)
solver = SolverFactory('ipopt')
results = solver.solve(orct_model, tee=True)
print(results)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       24
Number of nonzeros in inequality constraint Jacobian.:       80
Number of nonzeros in Lagrangian Hessian.............:      715

Total number of variables............................:       63
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       59
                     variables with only upper bounds:        0
Tot

In [5]:
print("Train error: ", accuracy_score(y_train, orct_predict(orct_model, X_train)))
print("Test error: ",accuracy_score(y_test, orct_predict(orct_model, X_test)))

Train error:  0.9910714285714286
Test error:  0.9473684210526315
